In [60]:
import pandas as pd
import json

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [33]:
supplements = pd.read_csv("Supplements.csv")
supplements.head(5)

,Name,Url
0,Abscess Root,https://www.webmd.com/vitamins/ai/ingredientmo...
1,Abuta,https://www.webmd.com/vitamins/ai/ingredientmo...
2,Acacia Rigidula,https://www.webmd.com/vitamins/ai/ingredientmo...
3,Acai,https://www.webmd.com/vitamins/ai/ingredientmo...
4,Acerola,https://www.webmd.com/vitamins/ai/ingredientmo...


In [34]:
supplements = supplements[supplements["Name"].str.startswith('Beta-Carotene')]
supplements.tail(5)

,Name,Url
133,Beta-Carotene,https://www.webmd.com/vitamins/ai/ingredientmo...


In [ ]:
def get_element_text(supplementName, elementSection, elementPath):

    try:
        element = wait_time.until(EC.presence_of_element_located((By.XPATH, elementPath)))
        text = element.text.strip()
    except Exception as e:
        text = f"No {elementSection} information for {supplementName}"
    finally:
        return text

In [71]:
def get_usess(elementPath):

    uses = {}

    try:
        element = wait_time.until(EC.presence_of_element_located((By.XPATH, elementPath)))
        h3_tags = element.find_elements(By.TAG_NAME, "h3")
        
        ul_counter = 1
        for h3 in h3_tags:
            uses_per_tag = []

            ul_path = f"/html/body/div[1]/div/main/div/div[2]/div/div/div[2]/div[1]/div[{ul_counter}]/ul"
            ul_element = element.find_elements(By.XPATH, ul_path)

            li_elements = ul_element.find_elements(By.TAG_NAME, "li")
            for item in li_elements:
                li_text = item.strip()
                uses_per_tag.append(li_text)

            uses[h3.text] = uses_per_tag
            ul_counter += 1

        #print(json.dumps(uses, indent=4))
        print(uses)
    except Exception as e:
        print(e)

In [86]:
def get_uses(elementPath):
    uses = {}

    try:
        element = wait_time.until(EC.presence_of_element_located((By.XPATH, elementPath)))
        h3_tags = element.find_elements(By.TAG_NAME, "h3")
        
        ul_counter = 1
        for h3 in h3_tags:
            uses_per_tag = []

            ul_path = f"/html/body/div[1]/div/main/div/div[2]/div/div/div[2]/div[1]/div[{ul_counter}]/ul"
            ul_elements = element.find_elements(By.XPATH, ul_path)

            if ul_elements:
                li_elements = ul_elements[0].find_elements(By.TAG_NAME, "li")
                
                for item in li_elements:
                    li_text = item.text.strip() 
                    uses_per_tag.append([li_text])

                uses[h3.text] = uses_per_tag
            ul_counter += 1

    except Exception as e:
        print(e)
    finally:
        return uses

In [ ]:
def get_interactions(elementPath):
    

In [89]:
supplement_info = {}

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

for _, info in supplements.head(1).iterrows():

    supplementName = info.Name
    supplementURL = info.Url

    driver.get(supplementURL)
    wait_time = WebDriverWait(driver,5)
    overview = get_element_text(supplementName, "Overview", "/html/body/div[1]/div/main/div/div[2]/div/div/div[1]/div[1]/div")
    uses = get_uses("/html/body/div[1]/div/main/div/div[2]/div/div/div[2]/div[1]")
    sideEffects = get_element_text(supplementName, "Side Effects", "/html/body/div[1]/div/main/div/div[2]/div/div/div[3]/div[1]")
    precautions = get_element_text(supplementName, "Precautions", "/html/body/div[1]/div/main/div/div[2]/div/div/div[4]/div[1]/div")
    interactons = []
    dosing = get_element_text(supplementName, "Dosing", "/html/body/div[1]/div/main/div/div[2]/div/div/div[6]/div[1]/div")

    supplement_info[supplementName] = {
        "Overview": overview,
        "Uses": uses,
        "SideEffects": sideEffects,
        "Precautions": precautions,
        "Interactions": interactons,
        "Dosing": dosing 
    }

driver.quit()

In [90]:
print(json.dumps(supplement_info, indent=4))

{
    "Beta-Carotene": {
        "Overview": "Beta-carotene belongs to a group of colored pigments called carotenoids. It's converted to vitamin A in the body and found in many fruits and vegetables.\n\nBeta-carotene and other red, orange, and yellow pigments called carotenoids are considered antioxidants. They provide about 50% of the vitamin A needed in the diet. Vitamin A is an essential nutrient. Its antioxidant and anti-inflammatory effects help to protect cells from damage.\n\nPeople use beta-carotene for an inherited disorder marked by sensitivity to light. It is also used to prevent certain cancers, heart disease, cataracts, aging skin, and many other purposes, but there is no good scientific evidence to support most of these uses.\n\nMany global health authorities recommend getting beta-carotene and other antioxidants from food instead of supplements.",
        "Uses": {
            "Effective for": [
                [
                    "An inherited disorder marked by sensi